In [ ]:
# Version (Handel Outlier)  
# ===============================================================   
# 0. Imports
# ===============================================================
import pandas as pd, numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline  import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import HuberRegressor, LogisticRegression
from feature_engine.outliers import Winsorizer
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [ ]:
# ===============================================================
# 1. Read file & Basic fields
# ===============================================================
df = pd.read_excel("window_breakage_data.xlsx")
df = df.drop(columns=["YldFrctn", "Yield", "YieldX", "PF Rowstate"])
df = df.dropna(subset=["Pass/Fail"])
df["PassFail_bin"] = df["Pass/Fail"].map({"Pass": 1, "Fail": 0})

feature_cols = ["Cut speed", "Edge Deletion rate", "Spacer Distance",
                "Glass Supplier",
                "Glass thickness", "Window Size", "Ambient Temp"]
num_cols = ["Cut speed", "Edge Deletion rate", "Spacer Distance",
            "Glass thickness", "Window Size", "Ambient Temp"]
cat_cols = ["Glass Supplier"]

X = df[feature_cols]
y_reg = df["Breakage Rate"]
y_cls = df["PassFail_bin"]

In [ ]:
# ===============================================================
# 2. Preprocess: Impute → Winsorize → Standardize
# ===============================================================
winsor = Winsorizer(
    capping_method="quantiles",
    tail="both",
    fold=0.01,
    variables=["Glass thickness"]        # Only keeps this feature
)

numeric_tf = Pipeline([
    ("win", winsor),                     # ① keep DataFrame
    ("imp", SimpleImputer(strategy="median")),  # ② now ndarray OKK
    ("sc",  StandardScaler())            # ③ standardize
])

cat_tf = Pipeline([
    ("imp", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(drop="first"))
])

preprocess = ColumnTransformer([
    ("num", numeric_tf, num_cols),
    ("cat", cat_tf, cat_cols)
])

In [ ]:
# ===============================================================
# 3. Robust Regression + Logistic
# ===============================================================
huber_reg = Pipeline([
    ("prep", preprocess),
    ("hub",  HuberRegressor())
]).fit(X, y_reg)

log_reg = Pipeline([
    ("prep", preprocess),
    ("clf",  LogisticRegression(max_iter=1000, class_weight="balanced"))
]).fit(X, y_cls)

# Coefficients & Intercept
feat_names = preprocess.get_feature_names_out()
coef_reg   = huber_reg.named_steps["hub"].coef_
inter_reg  = huber_reg.named_steps["hub"].intercept_

coef_cls   = log_reg.named_steps["clf"].coef_[0]
inter_cls  = log_reg.named_steps["clf"].intercept_[0]

feat_idx = {f: i for i, f in enumerate(feat_names)}
def c(name): return coef_reg[feat_idx[name]]
def b(name): return coef_cls[feat_idx[name]]

In [ ]:
# ===============================================================
# 4. Build Pyomo model (with standardization + practical constraints)
# ===============================================================
suppliers = ["A","B","C","D"]

# Build supplier→feature mapping
sup_feat = {s: [f for f in feat_names if f.endswith(f"Supplier {s}")][0]
            for s in suppliers[1:]}

# ------------------------------------------------------------------
# Access the *fitted* numeric pipeline inside the trained preprocess
# ------------------------------------------------------------------
num_pipe = huber_reg.named_steps["prep"].named_transformers_["num"]
scaler   = num_pipe.named_steps["sc"]      # this scaler is already fitted
mu       = scaler.mean_
sigma    = scaler.scale_
def z(name, var):
    idx = num_cols.index(name)
    return (var - mu[idx]) / sigma[idx]

# Exogenous constants
glas_thick = 0.50
win_size   = 70
ambient_c  = 22

m = pyo.ConcreteModel()
# decision vars
m.cut_speed = pyo.Var(bounds=(0.3, 3.0))
m.edge_rate = pyo.Var(bounds=(13.0, 18.0))
m.spacer    = pyo.Var(bounds=(2.5, 6.0))
m.sup = pyo.Var(suppliers, within=pyo.Binary)
m.one_sup = pyo.Constraint(expr=sum(m.sup[s] for s in suppliers) == 1)

# predicted breakage (robust linear)
m.y_pred = pyo.Expression(
    expr = inter_reg
         + c("num__Cut speed")          * z("Cut speed", m.cut_speed)
         + c("num__Edge Deletion rate") * z("Edge Deletion rate", m.edge_rate)
         + c("num__Spacer Distance")    * z("Spacer Distance", m.spacer)
         + c("num__Glass thickness")    * z("Glass thickness", glas_thick)
         + c("num__Window Size")        * z("Window Size", win_size)
         + c("num__Ambient Temp")       * z("Ambient Temp", ambient_c)
         + sum(c(sup_feat[s]) * m.sup[s] for s in suppliers[1:])
)

# Pass probability constraint
logit_expr = (
    inter_cls
  + b("num__Cut speed")          * z("Cut speed", m.cut_speed)
  + b("num__Edge Deletion rate") * z("Edge Deletion rate", m.edge_rate)
  + b("num__Spacer Distance")    * z("Spacer Distance", m.spacer)
  + b("num__Glass thickness")    * z("Glass thickness", glas_thick)
  + b("num__Window Size")        * z("Window Size", win_size)
  + b("num__Ambient Temp")       * z("Ambient Temp", ambient_c)
  + sum(b(sup_feat[s]) * m.sup[s] for s in suppliers[1:])
)
m.pass_prob = pyo.Constraint(expr=logit_expr >= np.log(0.9/0.1))

# Practical limits & penalties
m.min_speed = pyo.Constraint(expr=m.cut_speed >= 1.0)
m.max_spacer = pyo.Constraint(expr=m.spacer <= 5.0)

m.obj = pyo.Objective(
    expr = m.y_pred + 0.05*m.cut_speed + 0.02*m.edge_rate,
    sense=pyo.minimize
)

In [ ]:
# ===============================================================
# 5. Solve
# ===============================================================
solver = SolverFactory("glpk")   # or glpk / gurobi
res = solver.solve(m, tee=True)

print("Status:", res.solver.termination_condition)
print("Cut Speed:", m.cut_speed.value)
print("Edge Rate:", m.edge_rate.value)
print("Spacer:", m.spacer.value)
print("Supplier:", [s for s in suppliers if m.sup[s]() > 0.5][0])
print("Pred Breakage:", m.y_pred())

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /var/folders/rc/km87dnjn7wvcyywzwry25sdh0000gn/T/tmpxdd1zxho.glpk.raw
 --wglp /var/folders/rc/km87dnjn7wvcyywzwry25sdh0000gn/T/tmp4jjpyi4y.glpk.glp
 --cpxlp /var/folders/rc/km87dnjn7wvcyywzwry25sdh0000gn/T/tmpdw4vxn3t.pyomo.lp
Reading problem data from '/var/folders/rc/km87dnjn7wvcyywzwry25sdh0000gn/T/tmpdw4vxn3t.pyomo.lp'...
/var/folders/rc/km87dnjn7wvcyywzwry25sdh0000gn/T/tmpdw4vxn3t.pyomo.lp:49: warning: lower bound of variable 'x10' redefined
/var/folders/rc/km87dnjn7wvcyywzwry25sdh0000gn/T/tmpdw4vxn3t.pyomo.lp:49: warning: upper bound of variable 'x10' redefined
4 rows, 8 columns, 12 non-zeros
4 integer variables, all of which are binary
53 lines were read
Writing problem data to '/var/folders/rc/km87dnjn7wvcyywzwry25sdh0000gn/T/tmp4jjpyi4y.glpk.glp'...
42 lines were written
GLPK Integer Optimizer 5.0
4 rows, 8 columns, 12 non-zeros
4 integer variables, all of which are binary
Preprocessing...
2 ro

In [ ]:
# Apply back to LGBM for validation ---------------------------

import joblib, pandas as pd, numpy as np

# 0. load pipelines -------------------------------------------------
cls_pipe = joblib.load("lgbm_classifier.pkl")          # Already Pipeline
reg_obj  = joblib.load("lgbm_regressor.pkl")           # tuple or Pipeline

# --- if tuple, rebuild ---
from sklearn.pipeline import Pipeline
if isinstance(reg_obj, tuple):                         # (preprocess, model)
    reg_pipe = Pipeline([("preprocess", reg_obj[0]),
                         ("model",       reg_obj[1])])
else:
    reg_pipe = reg_obj                                 # already Pipeline


# 1. Get full column names and "categorical fields" ---------------------------
full_cols   = list(cls_pipe.named_steps["preprocess"].feature_names_in_)
cat_columns = cls_pipe.named_steps["preprocess"].transformers_[1][2]  # the 'cat' list

# 2. Construct prescription settings ------------------------------------------
opt_vals = {
    "Cut speed":         1.0,
    "Edge Deletion rate":13.0,
    "Spacer Distance":   2.5,          # or 5.0
    "Glass Supplier":    "Supplier D",
    "Glass thickness":   0.50,
    "Window Size":       70,
    "Ambient Temp":      22
}

row = {}
for col in full_cols:
    if col in opt_vals:                 # Has numerical value → Use numerical value
        row[col] = opt_vals[col]
    elif col in cat_columns:            # Categorical column → Fill "Unknown"
        row[col] = "Unknown"
    else:                               # Remaining numerical columns → NaN
        row[col] = np.nan

X_new = pd.DataFrame([row])

# 3. Predict -------------------------------------------------
pass_prob = cls_pipe.predict_proba(X_new)[0, 1]
pass_pred = cls_pipe.predict(X_new)[0]
break_pred = reg_pipe.predict(X_new)[0]

print(f"Pass probability : {pass_prob:.3f}")
print(f"Predicted label  : {'Pass' if pass_pred==1 else 'Fail'}")
print(f"Pred Breakage    : {break_pred:.2f} pieces/batch")

Pass probability : 0.995
Predicted label  : Pass
Pred Breakage    : 1.64 pieces/batch
